In [ ]:
# imports and loading
import pickle
import pandas as pd
from sympy import symbols, Eq, solve
import numpy as np

# Define the symbolic variables
mRNARatio, cActivator, KdRNAP, KdRNAPAct, RNAP, KeqOpening, cInhibitor = symbols(
    'mRNARatio cActivator KdRNAP KdRNAPAct RNAP KeqOpening cInhibitor'
)

# Create the equation (cleaned up the string formatting)
eq = Eq(mRNARatio, 
        ((cActivator*KdRNAP + KdRNAPAct)*(KdRNAP + RNAP + KeqOpening*RNAP))/
        ((1 + cActivator + cInhibitor)*KdRNAP*KdRNAPAct + 
         cActivator*KdRNAP*(1 + KeqOpening)*RNAP + 
         KdRNAPAct*(1 + KeqOpening)*RNAP))

# Create a function that takes all parameters and returns the mRNA ratio
def calculate_mRNA_ratio(c_activator, kd_rnap, kd_rnap_act, rnap, keq_opening, c_inhibitor):
    # Create a dictionary of values to substitute
    subs_dict = {
        cActivator: c_activator,
        KdRNAP: kd_rnap,
        KdRNAPAct: kd_rnap_act,
        RNAP: rnap,
        KeqOpening: keq_opening,
        cInhibitor: c_inhibitor
    }
    
    # Substitute the values and evaluate
    result = eq.rhs.subs(subs_dict)
    return float(result)

In [31]:
# settings
gene = 'b1101'

# load
directory = '../data/output/saved_gene_results/'+gene+'/'
input_cAct_cInh = pd.read_pickle(directory+'cAct_cInh.pkl')
constants = pd.read_pickle(directory+'constants.pkl')

# calculate *new* mRNA
news = []
for index, row in input_cAct_cInh.iterrows():
    new = calculate_mRNA_ratio(
        c_activator=row['cAct'],
        c_inhibitor=row['cInh'],
        kd_rnap=constants['KdRNAP'],
        kd_rnap_act=constants['KdRNAPAct'],
        rnap=constants['RNAP'],
        keq_opening=constants['KeqOpening'],  
    )
    news.append(new)
input_cAct_cInh['new_mRNA'] = news

input_cAct_cInh

,cAct,cInh,new_mRNA
p1k_01008,5.574630,31.239558,0.377217
p1k_00898,18.327345,23.436567,1.035488
p1k_00528,1.113602,2.710069,0.754691
p1k_00486,0.505082,0.346206,1.182413
p1k_00418,0.316910,0.714920,0.861919
...,...,...,...
p1k_00416,13.750529,29.801578,0.753580
p1k_00803,2.052397,6.129756,0.639422
p1k_00151,0.021934,5.983291,0.151507
p1k_00386,0.041117,2.727951,0.293284
